In [1]:
# import the necessary packages
# 1. SQL Integration
import mysql.connector
from mysql.connector import errorcode
# 2. Computer Vision
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import imutils
import dlib
import cv2
# 3. Analysis
from scipy.spatial import distance as dist
import numpy as np
import time
from datetime import datetime

In [2]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "122412112001",
    database = 'capstone'
)

In [3]:
print(mydb)

In [4]:
cursor = mydb.cursor()

In [5]:
#cursor.execute('CREATE TABLE brLog(timestamp float, blink_count int, blink_rate float, primary key(timestamp))')

In [6]:
hog_face_detector = dlib.get_frontal_face_detector()

In [7]:
dlib_facelandmark = dlib.shape_predictor("D:/Study/Uni/1. Capstone II/PyCharm Codes/shape_predictor_68_face_landmarks.dat")

In [8]:
# Test code for facial detection
"""
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = hog_face_detector(gray)
    for face in faces:

        face_landmarks = dlib_facelandmark(gray, face)

        for n in range(0, 68):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)


    cv2.imshow("Face Landmarks", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
"""

'\nwhile True:\n    _, frame = cap.read()\n    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n\n    faces = hog_face_detector(gray)\n    for face in faces:\n\n        face_landmarks = dlib_facelandmark(gray, face)\n\n        for n in range(0, 68):\n            x = face_landmarks.part(n).x\n            y = face_landmarks.part(n).y\n            cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)\n\n\n    cv2.imshow("Face Landmarks", frame)\n\n    key = cv2.waitKey(1)\n    if key == 27:\n        break\ncap.release()\ncv2.destroyAllWindows()\n'

In [9]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear

In [10]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [11]:
# start video capture
cap = cv2.VideoCapture(0)

# define two constants, 
# 1. the EAR threshold that the eye needs to pass (be lower than) in order to indicate a blink
# 2. number of frames the eyes have to be in while having EAR lower than the detection threshold
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

# eyes-closed frame counter & total number of blinks while the code is running
COUNTER = 0
TOTAL = 0

# create timer, with tick = frequency of capturing data into database
start_time = time.time()
tick = 2

# notification switch
notisw = True

# eye-strain & drowsiness notification variables
strnotif = ''
drwnotif = ''

#20-20-20 notification variable & flag
restnotif = ''
restFlag = False

#strain duration (counted by frames)
strain_cnt = 0

# initialize blinkrate variable & drowsiness flag
#      (differentiate between eye strain and just having some shut eye for relaxation)
blink_rate = 0
drowsy_flag = False

while True:
    current_time = time.time()
    duration = current_time - start_time
    _, frame = cap.read()
    
    #convert video capture to grayscale for detection model to work
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = hog_face_detector(gray)
    for face in faces:
        # conversion to NumPy array to work with
        face_landmarks = face_utils.shape_to_np(dlib_facelandmark(gray, face))

        leftEye = face_landmarks[lStart:lEnd]
        rightEye = face_landmarks[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2
        
        # calculate the convex hull for each eye & visualization for detection of each 
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        # check to see if the eye aspect ratio is below the blink threshold
        # if yes -> increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        else:
            # if the eyes were closed for a sufficient # of frames -> increment the total number of blinks
            if COUNTER >= EYE_AR_CONSEC_FRAMES and COUNTER < 120:
                TOTAL += 1
                # reset the eyes-closed frame counter
                COUNTER = 0
            elif COUNTER >= 12: #if eyes are closed for longer or equal to 12 frames (roughly 400ms), then it counts as drowsy
                drowsy_flag = True
                
            # reset the eyes-closed frame counter
            # COUNTER = 0
        
        
        # blink rate calculation
        if duration != float(0):
            blink_rate = 60*TOTAL/duration
        
        br = int(blink_rate)
        
        
        # notification system (on-frame)
        if notisw == True:
            if br > 20:
                strnotif = 'Your eye is straining'
            elif br > 15 and br <= 20:
                strnotif = 'Normal blink rate'
            elif br > 12 and br <= 15:
                strnotif = 'Computer blink rate'
                strain_cnt += 1

            #if the eye suffers lower bilnk rate due to computer use for more than 5 minutes -> alert the user to go get some rest
            # 5 minutes = 300 seconds. Considering webcam video is captured at 30fps -> eyes need to strain for 9000 frames
            if strain_cnt == 9000:
                strnotif = 'Go get some rest'
                notisw = False
        
        #drowsiness detection
        if br <= 12 & drowsy_flag == True:
            drwnotif = 'STOP AND GO TO SLEEP YOU SLEEPYHEAD'
        
        '''
        if int(duration) == tick:
            tick += 2
            cursor.execute('insert into brLog values (%s,%s,%s)', (duration, TOTAL, blink_rate))            
            mydb.commit()
        '''
        
        
        # 20-20-20 rule enforcement
        # after 20', a notification will show up alerting the user to take a rest
        if int(duration) % 5 == 0 and int(duration) != 0 and restFlag == False:
            restnotif = 'Take a rest for 20 seconds'
            restFlag = True
            
        
        # draw the total number of blinks on the frame along with
        # the computed eye aspect ratio for the frame
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Duration: {:.2f}".format(duration), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blink rate: {:.2f}".format(blink_rate), (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, strnotif, (50, 180),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, drwnotif, (50, 200),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        cv2.putText(frame,restnotif, (50, 250),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Face Landmarks", frame)

    key = cv2.waitKey(1)
    # stop code altogether & clean up window frame by pressing 1
    if key == 49:
        break
    # the user will need to confirm that they will take a rest under 20-20-20 rule by pressing 2
    elif key == 50:
        restnotif = ''
        restFlag = False
    # confirm eye strain rest by pressing 3
    elif key == 51:
        strain_cnt = 0
        notisw =  True
cap.release()
cv2.destroyAllWindows()

In [12]:
# Closing MySQL connection
mydb.close()